In [1005]:
import re
import pandas as pd 
import numpy as np

In [1006]:
Posts = pd.read_csv('../../160-Stackoverflow-Data/300000_rows/Posts.csv', \
                    dtype = {'LastEditorDisplayName': str, 'Tags': str})

In [1007]:
Posts.head()

,Id,PostTypeId,ParentID,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,CommunityOwnedDate,ClosedDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount
0,48870881,1,NaN,48871194.0,2018-02-19T17:05:48.193,1,50.0,<p>I have a json file with data as below:</p>\...,2804169.0,2804169.0,NaN,2018-02-19T18:42:53.487,2018-02-19T18:53:19.357,NaN,NaN,PHP search array return all values,<php><arrays>,2.0,0,NaN
1,48870882,1,NaN,48870993.0,2018-02-19T17:05:49.097,0,27.0,<p>I am currently working on a web application...,5128923.0,2598770.0,NaN,2018-02-19T17:13:33.920,2018-02-19T17:13:49.300,NaN,NaN,Creating a reactive user input tab,<c#><html><asp.net><.net><visual-studio>,2.0,0,NaN
2,48870883,2,48869547.0,NaN,2018-02-19T17:05:50.073,1,NaN,<p>Try to add hint to the query:</p>\r\n\r\n<p...,8214734.0,8214734.0,NaN,2018-02-20T10:20:21.007,2018-02-20T10:20:21.007,NaN,NaN,NaN,NaN,NaN,2,NaN
3,48870884,2,3865445.0,NaN,2018-02-19T17:05:50.273,0,NaN,In support of Sean's post:\r\n\r\n<p>The place...,2055283.0,NaN,NaN,NaN,2018-02-19T17:05:50.273,NaN,NaN,NaN,NaN,NaN,0,NaN
4,48870885,2,48870539.0,NaN,2018-02-19T17:05:51.300,0,NaN,<p>I discovered the convention (couldn't find ...,1052870.0,NaN,NaN,NaN,2018-02-19T17:05:51.300,NaN,NaN,NaN,NaN,NaN,1,NaN


## The number of answers against tags

In [1008]:
#tags of 'question' posts that the user posted 'answers' to, and count the number of the user's answers for each tag
def answers_user(user_id):
    """
    @param user_id: int 
    return: series 
    """
    filtered = []
    pattern = '[<>]'
    regex = re.compile(pattern, flags = re.IGNORECASE)
    answers_id = list(Posts.loc[(Posts.OwnerUserId == user_id) &  (Posts.PostTypeId == 2)]['ParentID'])

    for i in range(len(answers_id)):
        tags = Posts.loc[(Posts.Id == answers_id[i])]['Tags']
        try:
            filtered.extend(list(filter(None,regex.split(tags.iloc[0]))))
        except:
            continue
    freq_answers = pd.DataFrame({'Tag':filtered}).groupby('Tag').size()
    return freq_answers

In [1032]:
#example
answers_user(476)

pandas.core.series.Series

## The number of accepted answers against tags

In [1010]:
#tags of 'question' posts that the user's answer was accepted to('AcceptedAnswer'), and count the number of the user's accepted answers for each tag
def accepted_ans(user_id):
    """
    @param user_id: int 
    return: series 
    """
    accepted_tags = []
    pattern = '[<>]'

    regex = re.compile(pattern, flags = re.IGNORECASE)
    answers = Posts.loc[(Posts.OwnerUserId == user_id) &  (Posts.PostTypeId == 2)]

    for i in range(len(answers)):
        tags = Posts.loc[(Posts.AcceptedAnswerId == answers.Id.iloc[i])]['Tags']
        try:
            accepted_tags.extend(list(filter(None,regex.split(tags.iloc[0]))))
        except:
            continue
    freq_acanswers = pd.DataFrame({'Tag':accepted_tags}).groupby('Tag').size()
    return freq_acanswers

In [1011]:
#example
accepted_ans(5734311)

Tag
angular       1
angular5      1
javascript    1
dtype: int64

## The number of comments against tags

In [1012]:
Comments = pd.read_csv('../../160-Stackoverflow-Data/300000_rows/Comments.csv')

In [1013]:
Comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,85065265,49030489,4,Possible duplicate of [Node.js - require empty...,2018-02-28T13:16:41.767,5734311.0
1,85065268,49030285,0,The problem with this is the bad structure log...,2018-02-28T13:16:45.110,2718972.0
2,85065269,49030414,0,Can you elaborate a little more on ViewModel? ...,2018-02-28T13:16:50.880,9305895.0
3,85065270,49022014,1,In `rollapply` the `width=` can be a one compo...,2018-02-28T13:16:51.247,516548.0
4,85065271,49029381,0,"Yeah, I'm testing here and it doesnt seem to b...",2018-02-28T13:16:52.050,1850609.0


In [1014]:
#tags of 'question' posts that the user posted 'comments' to, and count the number of the user's comments for each tag
def comments_user(user_id):
    """
    @param user_id: int 
    return: series 
    """
    comments_tags = []
    pattern = '[<>]'


    regex = re.compile(pattern, flags = re.IGNORECASE)
    comments = Comments.loc[(Comments.UserId == user_id)]['PostId']

    for i in range(len(comments)):
        tags = Posts.loc[(Posts.Id == comments.iloc[i])]['Tags']
        try:
            comments_tags.extend(list(filter(None,regex.split(tags.iloc[0]))))
        except:
            continue
    freq_comments = pd.DataFrame({'Tag':comments_tags}).groupby('Tag').size()
    return freq_comments

In [1015]:
#example
comments_user(476)

Tag
.htaccess                8
ajax                     3
anaconda                 1
angular                  4
angularjs                1
api                      2
arrays                   3
autobahn                 1
calculation              1
character-encoding       2
chinese-locale           1
class                    3
css                      1
curl                     1
database                14
datetime                 4
dependency-injection     2
dictionary               4
dns                      1
ecmascript-6             3
ecomexpress              2
file                     1
file-upload              2
forms                    5
function                 2
generator                2
gravity-forms-plugin     1
html                    11
html5                    7
html5-video              1
                        ..
templates                1
timedelta                3
timezone                 6
transactions             1
twisted                  1
type-conversion         

## Total number of activities against tags

In [1016]:
#the number of activities such as answers, accepted answers and comments of the user for each tag
def activities(user_id):
    """
    @param user_id: int 
    return: dataframe
    """
    answers = answers_user(user_id)
    acc_answers = accepted_ans(user_id)
    comments = comments_user(user_id)
    user_table = pd.DataFrame({'Answers' : answers ,'Accepted Answers' : acc_answers , 'Comments': comments})
    user_table = user_table.fillna(0)
    if len(user_table) != 0 :
        user_table['Total'] = user_table.apply(lambda row: (row['Answers'])+(row['Accepted Answers'])+(row['Comments']), axis=1)
        
    return user_table

In [1017]:
#example
activities(476)

,Accepted Answers,Answers,Comments,Total
.htaccess,0.0,0.0,8.0,8.0
abstraction,1.0,1.0,0.0,2.0
access,1.0,1.0,0.0,2.0
ajax,0.0,0.0,3.0,3.0
anaconda,0.0,0.0,1.0,1.0
angular,1.0,1.0,4.0,6.0
angularjs,0.0,0.0,1.0,1.0
api,1.0,1.0,2.0,4.0
arrays,0.0,0.0,3.0,3.0
autobahn,0.0,0.0,1.0,1.0


## Tag Synonyms

In [1018]:
with open('../../160-Stackoverflow-Data/tags/all_unique_tags.txt', 'r') as f:
    tags_unique = f.readlines()
tags_unique = [i.rstrip() for i in tags_unique]

In [1021]:
TagSyn = pd.read_csv("../../160-Stackoverflow-Data/tags/TagSynonyms.csv")

In [1022]:
TagSyn.head()

,Id,SourceTagName,TargetTagName,CreationDate,OwnerUserId,AutoRenameCount,LastAutoRename,Score,ApprovedByUserId,ApprovalDate
0,3,windows-forms,winforms,2010-07-14 06:34:55,23354,584,2018-04-06 17:43:18,0,23354.0,2010-07-14 06:34:55
1,5,c-sharp,c#,2010-07-14 22:30:24,17174,178,2018-01-06 10:53:21,0,17174.0,2010-07-14 22:30:24
2,7,hg,mercurial,2010-07-15 22:46:06,17174,1001,2018-04-16 10:46:21,0,17174.0,2010-07-15 22:46:06
3,9,dotnet,.net,2010-07-16 00:02:29,17174,1323,2018-04-19 11:28:31,0,17174.0,2010-07-16 00:02:29
4,10,threading,multithreading,2010-07-16 00:04:00,17174,3844,2018-04-21 21:58:51,0,17174.0,2010-07-16 00:04:00


In [1023]:
tagcols = pd.Series(TagSyn.SourceTagName.values,TagSyn.TargetTagName).to_dict()

In [1024]:
tagcols

{'winforms': 'windows.forms',
 'c#': '.cs-file',
 'mercurial': 'hg',
 '.net': '.net-framework',
 'multithreading': 'cross-threading',
 'ruby-on-rails': 'ror',
 'javascript': 'javascript-module',
 'visual-c++': 'vc',
 'version-control': 'sccs',
 'blogs': 'blogging',
 'user-interface': 'gui',
 'visual-studio-2008': 'vs2008',
 'visual-studio-2005': 'vs2005',
 'visual-studio-2010': 'visual-studio-2010-beta-2',
 'primary-key': 'primary-keys',
 'foreign-keys': 'foreign-key-relationship',
 'postgresql': 'sql-postgres',
 'macos': 'osx',
 'java-me': 'jme',
 'internet-explorer': 'msie',
 'applet': 'japplet',
 'ejb-3.0': 'ejb-3',
 'glassfish-3': 'glassfish-v3',
 'java-5': 'jdk1.5',
 'java-6': 'jdk1.6',
 'java-7': 'jdk7',
 'java-ee': 'j2ee',
 'java-ee-5': 'jee-5',
 'java-ee-6': 'jee-6',
 'java-io': 'java.io',
 'jpa-2.0': 'jpa-2',
 'jsf-2': 'jsf2.0',
 'jsp-tags': 'jsp-tag-files',
 'jsp-fragments': 'jspf',
 'servlets': 'webservlet',
 'servlet-filters': 'servlet-filter',
 'tomcat-5.0': 'tomcat-5',
 '

In [1025]:
tagcols_inv = {value: key for key, value in tagcols.items()}

In [1026]:
tagcols_inv

{'windows.forms': 'winforms',
 '.cs-file': 'c#',
 'hg': 'mercurial',
 '.net-framework': '.net',
 'cross-threading': 'multithreading',
 'ror': 'ruby-on-rails',
 'javascript-module': 'javascript',
 'vc': 'visual-c++',
 'sccs': 'version-control',
 'blogging': 'blogs',
 'gui': 'user-interface',
 'vs2008': 'visual-studio-2008',
 'vs2005': 'visual-studio-2005',
 'visual-studio-2010-beta-2': 'visual-studio-2010',
 'primary-keys': 'primary-key',
 'foreign-key-relationship': 'foreign-keys',
 'sql-postgres': 'postgresql',
 'osx': 'macos',
 'jme': 'java-me',
 'msie': 'internet-explorer',
 'japplet': 'applet',
 'ejb-3': 'ejb-3.0',
 'glassfish-v3': 'glassfish-3',
 'jdk1.5': 'java-5',
 'jdk1.6': 'java-6',
 'jdk7': 'java-7',
 'j2ee': 'java-ee',
 'jee-5': 'java-ee-5',
 'jee-6': 'java-ee-6',
 'java.io': 'java-io',
 'jpa-2': 'jpa-2.0',
 'jsf2.0': 'jsf-2',
 'jsp-tag-files': 'jsp-tags',
 'jspf': 'jsp-fragments',
 'webservlet': 'servlets',
 'servlet-filter': 'servlet-filters',
 'tomcat-5': 'tomcat-5.0',
 '

In [1027]:
"osx" in tagcols.values()

True

In [1028]:
"blogs" in tagcols.keys()

True

In [1029]:
tagcols.get("osx")

In [1030]:
[target_tag for target_tag, source_tag in tagcols.items() if 'blogging' in source_tag][0]

'blogs'

In [1031]:
accepted_ans(476).rename({'php':'php'})

Tag
abstraction           1
access                1
angular               1
api                   1
backend               1
character-encoding    2
design-patterns       1
encoding              1
interface             1
iso-8859-1            1
mixins                1
operating-system      1
oracle                1
php                   5
postgresql            1
python                1
python-3.x            1
rest                  1
syntax                1
unicode               1
utf-8                 1
dtype: int64